In [ ]:
import tensorflow as tf
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])
product = tf.matmul(matrix1, matrix2)

sess = tf.Session()
result = sess.run(product)
print result
# ==> [[ 12.]]
sess.close()

with tf.Session() as sess:
    result = sess.run([product])
    print result
    


### with...Device 语句用来指派特定的 CPU 或 GPU 执行操作

* "/cpu:0" : 机器的 CPU.
* "/gpu:0" : 机器的第一个 GPU, 如果有的话.
* "/gpu:1" : 机器的第二个 GPU, 以此类推.

In [ ]:
with tf.Session() as sess:
    with tf.device("/gpu:1"):
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)

In [ ]:
# 进入一个交互式 TensorFlow 会话.
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 使用初始化器 initializer op 的 run() 方法初始化 'x' 
x.initializer.run()

# 增加一个减法 sub op, 从 'x' 减去 'a'. 运行减法 op, 输出结果 
sub = tf.sub(x, a)
print sub.eval()
# ==> [-2. -1.]

##  Variables
通常会将一个统计模型中的参数表示为一组变量. 例如, 可以将一个神经网络的权重作为某个变量存储在一个 tensor 中. 在训练过程中, 通过重复运行训练图, 更新这个 tensor.

In [ ]:
state = tf.Variable(0, name="counter")

# 创建一个 op, 其作用是使 state 增加 1
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 启动图后, 变量必须先经过`初始化` (init) op 初始化,
# 首先必须增加一个`初始化` op 到图中.
init_op = tf.initialize_all_variables()

# 启动图, 运行 op
with tf.Session() as sess:
    # 运行 'init' op
    sess.run(init_op)
    # 打印 'state' 的初始值
    print sess.run(state)
    # 运行 op, 更新 'state', 并打印 'state'
    for _ in range(3):
        sess.run(update)
        print sess.run(state)

# 输出:
# 0
# 1
# 2
# 3

#result = sess.run([mul, intermed])
#取回多個 tensor

### Feed

In [ ]:
input1 = tf.placeholder(tf.types.float32)
input2 = tf.placeholder(tf.types.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
    print sess.run([output], feed_dict={input1:[7.], input2:[2.]})

# 输出:
# [array([ 14.], dtype=float32)]

###  MNIST

In [ ]:
import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 28*28=784，这里的None表示此张量的第一个维度可以是任何长度的
x = tf.placeholder("float", [None, 784])  
# W代表權重，W的维度是[784，10]，因为我们想要用784维的图片向量乘以它以得到一个10维的证据值向量，每一位对应不同数字类
W = tf.Variable(tf.zeros([784,10]))
# b代表偏移量
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x,W) + b)
# y_ 是實際的分佈
y_ = tf.placeholder("float", [None,10])
# cross_entropy 用来衡量我们的预测用于描述真相的低效性
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# 梯度下降算法（gradient descent algorithm）以0.01的学习速率最小化交叉熵
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# 设置好了我们的模型後。在运行计算之前，我们需要添加一个操作来初始化我们创建的变量
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# 让模型循环训练1000次
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
# 評估模型效能
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})